In [ ]:
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
import glob
import os
import tensorflow as tf
print(tf.__version__)
import numpy as np 
import cv2
import shutil
import stat
import keras

img_width = 128
img_height = 128

spec_width = 128
spec_height = 128

num_imgs = 30

model = keras.models.load_model('Magician.h5')
fr_model = keras.models.load_model('Face_recognition.h5')

img_ext = ["jpg","png", "jpeg"];
sound_ext = ["wav"];
ext = img_ext + sound_ext

path = 'C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data/False/'

In [ ]:
def load_img(path):
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (img_width, img_height))
    img = np.array(img) / 256.0
    return img

fr_model = keras.models.load_model('Face_recognition.h5')

def load_audio(path):
    audio = Audio.from_file(path).trim(0.05,2.05)
    
    spectrogram = Spectrogram.from_audio(audio)
    
    image = spectrogram.to_image(shape=(spec_width, spec_height),invert=False)

    return np.array(image) / 256

def prep_imgs(imgs):
    length = len(imgs)
    if (length > num_imgs):
        n = length - num_imgs
        i = (int)(length / (n + 1))
        for l in range(1, n + 1):
            imgs.pop(length - i * l)
    elif (len(imgs) < num_imgs):
        n =  num_imgs - length
        i = (int)(length / (n + 1))
        for l in range(1, n + 1):
            imgs.insert(length - i * l, imgs[length - i * l - 1])
    return imgs

def predict(path):
    imgs = []
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        if (os.path.splitext(files[0])[1].split(".")[1] in sound_ext):
            audio = load_audio(files[0])
        else:
            for img in files: 
                imgs.append(fr_model.predict(load_img(img).reshape(1, img_width, img_height, 3), verbose = 0))
    imgs = prep_imgs(imgs)
    imgs = np.array(imgs).reshape(1, num_imgs)
    audio = audio.reshape(1, spec_width, spec_height)
    return model.predict([imgs, audio], verbose = 0)

In [ ]:
for it in os.scandir(path):
    print(predict(it.path))